In [38]:
%matplotlib inline

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import re

In [40]:
def IR_Flag(rate):
    if rate > .12:
        return True
    else:
        return False

In [41]:
def num_strip_rt(data):
    if '%' in data:
        data = data.rstrip('%')
        
    if "." in data:
        data = float(data)/100
        
    return data
    

In [42]:
def get_int_from_text(data):
#     data = re.search('[0-9]*', data).group(0)
    data = re.findall('[0-9]+', data) # produces a 1 item list
    
#     print("data is type {}".format(type(data)))
    
#     data = [x for x in data if x != '']

    try:
        return data[0]
    except: 
        return -1
    
    
def own(data):
    if data in ['MORTGAGE','OWN']:
        return 1
    else:
        return 0
                     

In [43]:
# get loandata
raw_data = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')

In [44]:
raw_data.dropna(inplace=True) # drop the na rows
# clean fico range and get score 
raw_data['FICO.Score'] = map(lambda x: int(x.split('-')[0]), raw_data['FICO.Range']) # clean fico range and get score #ask Kyle why this runs so slow

raw_data['Debt.To.Income.Ratio'] = [num_strip_rt(data) for index, data in raw_data['Debt.To.Income.Ratio'].iteritems()]
raw_data['Interest.Rate'] = [num_strip_rt(data) for index, data in raw_data['Interest.Rate'].iteritems()]
raw_data['Loan.Length'] = [get_int_from_text(data) for index, data in raw_data['Loan.Length'].iteritems()]
raw_data['Employment.Length2'] = [get_int_from_text(data) for index, data in raw_data['Employment.Length'].iteritems()]
raw_data = raw_data[raw_data['Employment.Length2'] > 0] # delete any employment length -1 generated by original N/A
raw_data['Loan.Length'] = raw_data['Loan.Length'].astype(int)
 # add rate flag
raw_data['IR_TF'] = [IR_Flag(rate) for index, rate in raw_data['Interest.Rate'].iteritems()]
raw_data['OWN'] = [own(desc) for index, desc in raw_data['Home.Ownership'].iteritems()]


In [52]:

gbhome = raw_data.groupby('Home.Ownership')

In [24]:
raw_data.ix[raw_data['Employment.Length2'] == -1]

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,Employment.Length2
67629,16000,15925.00,0.0762,36,debt_consolidation,0.0648,IL,MORTGAGE,6000.00,765-769,8,19539,0,n/a,-1
51929,9525,9500.00,0.1581,36,small_business,0.1135,CA,RENT,2666.67,670-674,15,5610,1,n/a,-1
84563,8000,8000.00,0.0762,36,other,0,NC,MORTGAGE,3500.00,765-769,7,0,1,n/a,-1
64884,9000,9000.00,0.0603,36,vacation,0.0558,NJ,MORTGAGE,9583.33,815-819,11,675,0,n/a,-1
85644,10000,10000.00,0.1016,36,debt_consolidation,0.093,TX,MORTGAGE,3333.33,725-729,8,8940,2,n/a,-1
19548,4000,4000.00,0.1074,36,debt_consolidation,0.0197,KS,RENT,2900.00,695-699,13,1869,1,n/a,-1
45941,3000,3000.00,0.0976,36,debt_consolidation,0.2182,VA,OWN,1100.00,720-724,4,2413,0,n/a,-1
33161,3000,3000.00,0.1727,36,credit_card,0.1662,SC,RENT,3833.33,660-664,4,4136,0,n/a,-1
51833,4200,4200.00,0.0890,36,debt_consolidation,0.1043,CA,MORTGAGE,2254.00,700-704,7,4490,0,n/a,-1
84110,2500,2500.00,0.1409,36,medical,0.2095,NY,RENT,2666.67,665-669,9,15787,0,n/a,-1


In [25]:
ind_vars = list(raw_data.columns.values)

In [26]:
ind_vars


['Amount.Requested',
 'Amount.Funded.By.Investors',
 'Interest.Rate',
 'Loan.Length',
 'Loan.Purpose',
 'Debt.To.Income.Ratio',
 'State',
 'Home.Ownership',
 'Monthly.Income',
 'FICO.Range',
 'Open.CREDIT.Lines',
 'Revolving.CREDIT.Balance',
 'Inquiries.in.the.Last.6.Months',
 'Employment.Length',
 'Employment.Length2']

In [58]:
ind_vars = ['Amount.Requested',
 'Interest.Rate',
'Constant']

In [56]:
ind_vars

['Amount.Requested', 'Amount.Funded.By.Investors', 'Interest.Rate', 'Constant']

In [57]:
raw_data[ind_vars]

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Constant
81174,20000,20000.00,0.0890,1
99592,19200,19200.00,0.1212,1
80059,35000,35000.00,0.2198,1
15825,10000,9975.00,0.0999,1
33182,12000,12000.00,0.1171,1
62403,6000,6000.00,0.1531,1
48808,10000,10000.00,0.0790,1
22090,33500,33450.00,0.1714,1
76404,14675,14675.00,0.1433,1
15867,7000,7000.00,0.0691,1


In [55]:
raw_data['IR_TF']


81174     False
99592      True
80059      True
15825     False
33182     False
62403      True
48808     False
22090      True
76404      True
15867     False
94971      True
36911      True
41200      True
83869     False
53853     False
21399      True
62127      True
23446     False
44987     False
17977     False
86099      True
99483      True
28798      True
24168     False
10356      True
46027     False
2238       True
65278      True
4227      False
50182     False
          ...  
84265      True
80231     False
49533      True
102514     True
78618      True
86953      True
80129      True
85216     False
38247     False
91245      True
53041     False
63051     False
14446     False
68628      True
98758     False
13070     False
45836      True
52330      True
48243      True
63256      True
42124     False
78043     False
925       False
74047      True
49957      True
23735      True
65882      True
55610      True
38576      True
3116       True
Name: IR_TF, dtype: bool

In [39]:
 # add constant
raw_data['Constant'] = 1

In [ ]:
type(raw_data['Loan.Length'][:1])

In [59]:
logit = sm.Logit(raw_data['IR_TF'], raw_data[ind_vars])

In [60]:
result = logit.fit()

         Current function value: 0.000000
         Iterations: 35


C:\Users\bob071988\Anaconda2\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [61]:
coeff = result.params
print(coeff)

Amount.Requested       -0.000020
Interest.Rate       25575.419020
Constant            -3081.784218
dtype: float64


In [64]:
result.predict([10000,.16,1])

array([ 1.])

In [ ]:
def fix_data(raw_data):
       
    # clean interest rate using regex - ability to replace so df objects are mutable
    raw_data['Interest.Rate'] = [float(re.search('^[0-9]*.[0-9]*',loan).group(0))/100 for loan in raw_data['Interest.Rate']]

    # clean fico range and get score 
    raw_data['FICO.Score'] = map(lambda x: int(x.split('-')[0]), raw_data['FICO.Range'])
    
    # clean loan length
    raw_data['Loan.Length'] = map(lambda x: int(x.split()[0]), raw_data['Loan.Length'])
    
    # clean debt to income ratio
    raw_data['Debt.To.Income.Ratio'] = [float(re.search('^[0-9]*.[0-9]*',loan).group(0))/100 for loan in raw_data['Debt.To.Income.Ratio']]
    
    # add rate flag
    loan_data['IR_TF'] = [IR_Flag(rate) for index, rate in loan_data['Interest.Rate'].iteritems()]
    
    # add constant
    loan_data['Constant'] = [1 for row in loan_data.iterrows()]
    
    return raw_data

In [ ]:
raw_data.head()

In [ ]:
 # clean debt to income ratio
raw_data['Debt.To.Income.Ratio'] = [float(re.search('^[0-9]*.[0-9]*',loan).group(0) for loan in raw_data['Debt.To.Income.Ratio']]
    

In [ ]:
raw_data.head()

In [ ]:
raw_data.head()

In [24]:
loan_data = fix_data(raw_data)

ValueError: invalid literal for float(): 4%

In [8]:
raw_data.head()

In [10]:
ind_vars = list(loan_data.columns.values)

In [12]:
ind_vars.remove('Interest.Rate')

In [16]:
ind_vars.remove('IR_TF')

ValueError: list.remove(x): x not in list

In [ ]:
ind_vars.remove('Constant')

In [14]:
ind_vars

['Amount.Requested',
 'Amount.Funded.By.Investors',
 'Loan.Length',
 'Loan.Purpose',
 'Debt.To.Income.Ratio',
 'State',
 'Home.Ownership',
 'Monthly.Income',
 'FICO.Range',
 'Open.CREDIT.Lines',
 'Revolving.CREDIT.Balance',
 'Inquiries.in.the.Last.6.Months',
 'Employment.Length',
 'FICO.Score',
 'Constant']